In [ ]:
from dotenv import load_dotenv
from truststore import inject_into_ssl

dotenv_loaded = load_dotenv()
inject_into_ssl()

In [ ]:
from typing import Annotated

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [ ]:
from langfuse.callback import CallbackHandler

langfuse_handler = CallbackHandler()

In [ ]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4o")


def chatbot(state: State):
    return {"messages": [chat_model.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile().with_config({"callbacks": [langfuse_handler]})

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
from langchain.schema import HumanMessage


def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [HumanMessage(content=user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    stream_graph_updates(user_input)

In [ ]:
from langchain_community.tools import SearxSearchResults
from langchain_community.utilities import SearxSearchWrapper

search = SearxSearchWrapper()
muenchen_tool = SearxSearchResults(
    name="muenchen.de Search", wrapper=search, kwargs={"engines": ["google", "bing"], "query_suffix": "site:muenchen.de", "num_results": 3}
)

tourismus_tool = SearxSearchResults(
    name="muenchen.travel Search",
    wrapper=search,
    kwargs={"engines": ["google", "bing"], "query_suffix": "site:muenchen.travel", "num_results": 3},
)

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

tools = [muenchen_tool, tourismus_tool]

chat_model_with_tools = chat_model.bind_tools(tools)


def chatbot_with_tools(state: State):
    return {"messages": [chat_model_with_tools.invoke(state["messages"])]}


graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot_with_tools)
tool_node = ToolNode(tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
tool_graph = graph_builder.compile().with_config({"callbacks": [langfuse_handler]})

In [ ]:
display(Image(tool_graph.get_graph().draw_mermaid_png()))